In [3]:
import tensorflow as tf
from tensorflow.keras import layers,models,optimizers
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [6]:
(x_train,y_train),(x_test,y_test)=mnist.load_data()
x_train=x_train.astype('float32')/255.0
x_test=x_test.astype('float32')/255.0

In [7]:
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

In [8]:
#load a pre trained model MobileNetV2 with classification layer available
base_model=tf.keras.applications.MobileNetV2(input_shape=(224,224,3),include_top=False,weights='imagenet')

9406464/9406464 [==============================] - 0s 0us/step


In [11]:
model=models.Sequential()
model.add(layers.Reshape((28,28,1),input_shape=(28,28)))
model.add(layers.Conv2D(32,(3,3),activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(256,activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10,activation='softmax'))

In [12]:
#freezing the pre trained layers
for layer in base_model.layers:
    layer.trainable=False

In [14]:
#compile the model
model.compile(optimizer=optimizers.Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])

In [29]:
#custom training loop
epochs=10
batch_size=32
#create tf.data.Dataset fro training and activation
train_dataset=tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(60000).batch(batch_size)
val_dataset=tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(batch_size)
for epoch in range(epochs):
  print(f"Epoch{epoch+1}/{epochs}")
    #training loop
  for images,labels in train_dataset:
    with tf.GradientTape() as tape:
      predictions=model(images)
      loss=tf.keras.losses.categorical_crossentropy(labels,predictions)
    gradients=tape.gradient(loss,model.trainable_variables)
    optimizer=optimizers.Adam(learning_rate=0.001)
    optimizer.apply_gradients(zip(gradients,model.trainable_variables))
  #validation loop
  accuracy=tf.metrics.CategoricalAccuracy()
  for images,labels in val_dataset:
    predictions=model(images)
    accuracy.update_state(labels,predictions)
    val_acc=accuracy.result().numpy()
    print(f"Validation Accuracy:{val_acc}")


Epoch1/10
Validation Accuracy:0.96875
Validation Accuracy:0.984375
Validation Accuracy:0.9895833134651184
Validation Accuracy:0.9921875
Validation Accuracy:0.9937499761581421
Validation Accuracy:0.9947916865348816
Validation Accuracy:0.9910714030265808
Validation Accuracy:0.98828125
Validation Accuracy:0.9861111044883728
Validation Accuracy:0.987500011920929
Validation Accuracy:0.9801136255264282
Validation Accuracy:0.9817708134651184
Validation Accuracy:0.9831730723381042
Validation Accuracy:0.9799107313156128
Validation Accuracy:0.9750000238418579
Validation Accuracy:0.97265625
Validation Accuracy:0.9724264740943909
Validation Accuracy:0.9722222089767456
Validation Accuracy:0.9703947305679321
Validation Accuracy:0.96875
Validation Accuracy:0.96875
Validation Accuracy:0.96875
Validation Accuracy:0.967391312122345
Validation Accuracy:0.9661458134651184
Validation Accuracy:0.9674999713897705
Validation Accuracy:0.9675480723381042
Validation Accuracy:0.9675925970077515
Validation Accurac